# Create subjects from a list

Register users for DataONE from a `dict` of ORCiD URLs with subject names

In [ ]:
from d1_client.cnclient import CoordinatingNodeClient
from d1_common.types.dataoneTypes import Subject, person
from os import environ

AUTH_TOKEN = {
    'prod': 'TOKEN',
    'stage': 'TOKEN',
    'sandbox': 'TOKEN',
}
CN_URL = {
    'prod': 'https://cn.dataone.org/cn',
    'stage': 'https://cn-stage.test.dataone.org/cn',
    'sandbox': 'https://cn-sandbox.test.dataone.org/cn',
}
# The environment to use (change this to 'prod' to run on the production CN)
CN = 'prod'

environ['D1_AUTH_TOKEN'] = AUTH_TOKEN[CN]
auth_token = environ.get('D1_AUTH_TOKEN')

# Set the token in the request header
options: dict = {"headers": {"Authorization": "Bearer " + str(auth_token)}}
# Create the Member Node Client
client: CoordinatingNodeClient = CoordinatingNodeClient(CN_URL[CN], **options)

## Test user
If you've set your auth token correctly for the given `CN` environment, this should succeed in authenticating but fail to create the user (Ian already exists in the database)

In [ ]:
id_dict = {'http://orcid.org/0000-0001-5828-6070': ['Ian Nesbitt', 'nesbitt@nceas.ucsb.edu']}
for sid in id_dict:
    name = id_dict[sid][0]
    name_list = name.split(' ')
    given_name = id_dict[sid][0].split()[:-1]
    family_name = id_dict[sid][0].split()[-1]
    email = id_dict[sid][1]
    s = Subject(sid)
    p = person()
    p.subject = s
    p.givenName = given_name
    p.familyName = family_name
    p.mail = email
    try:
        client.registerAccount(p)
    except Exception as e:
        err_n = str(e).split('\n')[0]
        err_c = str(e).split('\n')[1]
        err_d = str(e).split('\n')[3]
        print('Error processing %s (%s)\n%s\n%s\n%s' % (name, sid, err_n, err_c, err_d))

Error processing Ian Nesbitt (http://orcid.org/0000-0001-5828-6070)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0001-5828-6070,dc=dataone,dc=org already exists


# List of users to create
```python
# format for list of users:
{
    'orcid': ['full name', 'email'],
    'orcid2': ['full name2', 'email2']
}
```

In [10]:
id_list = {
    'http://orcid.org/0000-0003-1338-2530': ['Aakash Kumar', 'aakash.kumar@stonybrook.edu'],
}

## Create users from list

In [11]:
for orcid in id_list:
    name = id_list[orcid][0]
    nameL = name.split(' ')
    givenNameL = id_list[orcid][0].split()[:-1]
    familyName = id_list[orcid][0].split()[-1]
    email = id_list[orcid][1]
    s = Subject(orcid)
    p = person()
    p.subject = s
    p.givenName = givenNameL
    p.familyName = familyName
    p.mail = email
    p.verified = True
    
    n = 1
    listLen = len(id_list[orcid])
    # while n < listLen:
    #     p.subject = id_list[orcid][n]
    #     n += 1
    print('%s %s %s' % (p.content()[0].value(), p.content()[1], p.content()[2]))
    if True:
        try:
            client.registerAccount(p)
        except Exception as e:
            try:
                err_n = str(e).split('\n')[0]
                err_c = str(e).split('\n')[1]
                err_d = str(e).split('\n')[3]
                print('Error processing %s (%s)\n%s\n%s\n%s' % (name, orcid, err_n, err_c, err_d))
            except:
                print(e)

http://orcid.org/0000-0003-1338-2530 ['Aakash'] Kumar


# Check that a user exists in the database
This runs a database search for the last user in the `for` loop above. A successful run will return a `SubjectInfo` object.

In [17]:
srch = client.getSubjectInfo(orcid)
srch.content()[0].content()

 'Aakash',
 'Kumar',
 0]

This returns the ORCiD of user returned by the above search. It should match the last one in the `id_list` above.

In [18]:
srch.content()[0].content()[0].value()

'http://orcid.org/0000-0003-1338-2530'